<a href="https://colab.research.google.com/github/navrgithub/NLP_Authorship_Attribution/blob/main/RNN_task3_morphed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP group project/data')

In [ ]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [ ]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',tokenizer_language='en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('final_task3_data.csv')

In [ ]:
len(df)

10661

In [ ]:
df.head()

,Generation,label
0,latest headlines on cnn business tl;dr the u.s...,ctrl
1,china wants to take a victory lap over its han...,ctrl
2,coronavirus disinformation creates challenges ...,ctrl
3,china coronavirus: eating wild animals made il...,ctrl
4,china's economy could shrink for the first tim...,ctrl


In [ ]:
df[['text']] = df[['text']]. astype(str)


In [ ]:
df = df[['text','label']]


In [ ]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm',  include_lengths = True)
LABEL = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)

pt = [TEXT.preprocess(x) for x in df['text']]


# load fastext simple embedding with 300d
TEXT.build_vocab(
    pt, 
    max_size = 25000,
    vectors='glove.6B.50d',
    unk_init = torch.Tensor.normal_
)

In [ ]:
label_to_index= {label: index for index, label in enumerate(df['label'].unique())}

In [ ]:
df['label_new'] = df['label'].apply(lambda y: label_to_index[y])

class DataFrameDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                data.Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], fields)
     

In [ ]:
df_new=df[['text','label_new']]


In [ ]:
examples=[]
for i,row in df_new.iterrows():
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))

In [ ]:
itrr=0
for i in examples:
  if type(i.label) != int:
    itrr+=1
     
     

In [ ]:
dataset = data.Dataset(examples, fields=[('text', TEXT), ('label', LABEL)])

In [ ]:
train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.65, 0.15, 0.2], stratified=True, strata_field='label')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)
# device = 'cpu'
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    datasets=(train_dataset, test_dataset, valid_dataset), 
    batch_size=32,
    sort_within_batch = True,
    sort_key=lambda x: len(x.text),
    device=device
)

We are working with  cuda


In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 256
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = False
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,713,359 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([30002, 50])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.7372,  0.3910,  0.5158],
        [ 0.1825, -0.0737,  0.3147,  ...,  1.3226, -2.6796, -0.1297],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.4447,  0.3625,  0.0473,  ..., -0.7721, -1.5739,  0.8893],
        [-1.0916, -0.6359,  0.8588,  ...,  0.9888,  0.3444, -0.6899],
        [ 1.2178, -1.4262,  0.5219,  ..., -1.3257,  0.9352, -2.3603]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.4447,  0.3625,  0.0473,  ..., -0.7721, -1.5739,  0.8893],
        [-1.0916, -0.6359,  0.8588,  ...,  0.9888,  0.3444, -0.6899],
        [ 1.2178, -1.4262,  0.5219,  ..., -1.3257,  0.9352, -2.3603]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss()

# device='cpu'
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # get the index of the highest prediction for each instance
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        # print(batch.text)
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        target = batch.label
        target = target.type(torch.LongTensor)

        # print(target)

        # print(predictions)
        
        loss = criterion(predictions.to(device), target.to(device))
        
        acc = multiclass_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            target = batch.label
            target = target.type(torch.LongTensor)
            
            loss = criterion(predictions.to(device), target.to(device))
            
            acc = multiclass_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()



    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 50
path = F"rnn_model_task3.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 2.097 | Train Acc: 25.33%
	 Val. Loss: 1.690 |  Val. Acc: 51.43%
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 1.982 | Train Acc: 28.14%
	 Val. Loss: 2.309 |  Val. Acc: 21.21%
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 2.454 | Train Acc: 10.60%
	 Val. Loss: 2.321 |  Val. Acc: 13.57%
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 2.395 | Train Acc: 10.34%
	 Val. Loss: 2.284 |  Val. Acc: 21.21%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 2.324 | Train Acc: 11.86%
	 Val. Loss: 2.188 |  Val. Acc: 22.26%
Epoch: 06 | Epoch Time: 0m 8s
	Train Loss: 2.231 | Train Acc: 15.28%
	 Val. Loss: 2.034 |  Val. Acc: 30.88%
Epoch: 07 | Epoch Time: 0m 9s
	Train Loss: 2.054 | Train Acc: 21.61%
	 Val. Loss: 1.791 |  Val. Acc: 38.51%
Epoch: 08 | Epoch Time: 0m 7s
	Train Loss: 2.051 | Train Acc: 21.45%
	 Val. Loss: 1.768 |  Val. Acc: 34.86%
Epoch: 09 | Epoch Time: 0m 8s
	Train Loss: 1.997 | Train Acc: 22.72%
	 Val. Loss: 1.945 |  Val. Acc: 30.39%
Epoch: 10 | Epoch Time: 0m 

In [ ]:
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.078 | Test Acc: 80.12%
